<h1> Trim T & S related files to include only data that is also associated with a BGC measurement </h1>

In [19]:
##Find indexes of T&S mean/residual files where there is BGC data and create new T&S data files to complete BGC Mapping.

#initialize
import netCDF4
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import scipy.io as sio
import pandas as pd
#import matplotlib as plt

#file paths
root = '/global/homes/k/kefalc/code/data_files/Final/'
mean_root = '/global/homes/k/kefalc/code/anom_files/'
savepath1 = '/global/homes/k/kefalc/code/bgc_anom_files/'

In [20]:
##SET FILE NAMES
bgc_file = '/global/homes/k/kefalc/code/data_files/Final/floats_years_2015_2019_temperature_salinity_oxygen_10daily_SO_30Skeep_random_floats_15perc.nc'
ts_file = '/global/homes/k/kefalc/code/data_files/Final/floats_years_2015_2019_temperature_salinity_oxygen_10daily_SO_30S_edit.nc'
time_i = (np.datetime64('2015-01-01') - np.datetime64('1970-01-01T00:00:00', 'ns')) / np.timedelta64(1, 'ns') / 1e9 / 86400.0
time_f = (np.datetime64('2019-12-31') - np.datetime64('1970-01-01T00:00:00', 'ns')) / np.timedelta64(1, 'ns') / 1e9 / 86400.0

testn = 0 #test number

In [21]:
#Open original BGC Data file
db = xr.open_dataset(bgc_file)

#remove NaNs
#db = db.dropna(dim="profile", how = "any")
#db = db.isel(~np.isnan(db.profile))

display(db)
#db.to_netcdf(root + 'floats_years_2015_2017_temperature_salinity_oxygen_10daily_SO_Testkeep_random_floats_25perc_edit.nc')
#print(np.count_nonzero(~np.isnan(db.profile)))
#profile dimension has length 2878

<xarray.Dataset>
Dimensions:      (pressure: 8, profile: 36506)
Coordinates:
    lat          (profile) float64 ...
    lon          (profile) float64 ...
  * pressure     (pressure) float64 -5.0 -10.0 -100.0 ... -1e+03 -1.5e+03 -3e+03
    time         (profile) float64 ...
    floatid      (profile) float64 ...
Dimensions without coordinates: profile
Data variables:
    temperature  (pressure, profile) float64 ...
    salinity     (pressure, profile) float64 ...
    oxygen       (pressure, profile) float64 ...
Attributes:
    contact:      Kristen Falcinelli, kefalc@uw.edu
    source_file:  /global/cfs/cdirs/m4003/maltrud/southernOceanRefined/profil...
    script:       /global/homes/k/kefalc/code/gpr-mapping-data/save_floats_E3...

In [22]:
#TS Data file
ds = xr.open_dataset(ts_file)
#ds = ds.dropna(dim="profile", how = "any")
display(ds)

<xarray.Dataset>
Dimensions:      (pressure: 8, profile: 239273)
Coordinates:
    lat          (profile) float64 ...
    lon          (profile) float64 ...
  * pressure     (pressure) float64 -5.0 -10.0 -100.0 ... -1e+03 -1.5e+03 -3e+03
    time         (profile) float64 ...
    floatid      (profile) float64 ...
Dimensions without coordinates: profile
Data variables:
    temperature  (pressure, profile) float64 ...
    salinity     (pressure, profile) float64 ...
    oxygen       (pressure, profile) float64 ...
Attributes:
    contact:      Kristen Falcinelli, kefalc@uw.edu
    source_file:  /global/cfs/cdirs/m4003/maltrud/southernOceanRefined/profil...
    script:       /global/homes/k/kefalc/code/gpr-mapping-data/save_floats_E3...

In [23]:
##Match new TS file with data from the BGC file
#create new indexes
lat1 = ds["lat"].to_index()
lon1 = ds["lon"].to_index()
lat2 = db["lat"].to_index()
lon2 = db["lon"].to_index()

index_ts = []
for i in range(len(lat2)):
    index_ts = np.append(index_ts, np.where((lat1 == lat2[i])# & (lon1 == lon2[i])
                                     ))
display(index_ts)
display(len(index_ts)) #2877
len(np.unique(index_ts))

index_ts = index_ts.astype(int)

array([   790.,   2103.,   3432., ..., 235610., 236902., 238218.])

36506

TS Data File

TS Residual File

In [ ]:
resfile = 'residualfile_plevel3_dates16436.0_18261.0_test0.nc'
result =  'forbgc_residualfile_plevel3_dates16436.0_18261.0_test0.nc'

In [8]:
##Match Indexes to the TS MEAN/RESIDUAL files to select that data and create a file to use in mapping BGC variables
for plevel in range (0,7):
    resfile = 'residualfile_plevel'+str(plevel)+'_dates'+str(time_i)+'_'+str(time_f)+'_test0.nc'
    dm = xr.open_dataset(mean_root + resfile)
    #display(dm)
    dm_new = dm.isel(i = index_ts)
    dm_new = dm_new.squeeze('pressure')
    #dm_new = dm_new.drop_dims('pressure')
    display(dm_new)

    dm_new.to_netcdf(root + 'forbgc_residualfile_plevel'+str(plevel)+'_dates'+str(time_i)+'_'+str(time_f)+'_test0.nc')

<xarray.Dataset>
Dimensions:         (i: 36506)
Coordinates:
    time            (i) datetime64[ns] ...
    latitude        (i) float64 ...
    longitude       (i) float64 ...
    pressure        int64 -5
Dimensions without coordinates: i
Data variables:
    mean_temp       (i) float64 ...
    mean_sal        (i) float64 ...
    residuals_temp  (i) float64 ...
    residuals_sal   (i) float64 ...
Attributes:
    description:          Estimated mean temperature and salinity, and residu...
    pressureindex:        0
    number_of_harmonics:  2
    window_size1:         700000.0
    window_size2:         700000.0
    filename:             /global/homes/k/kefalc/code/anom_files/test/residua...
    created_date:         2025-05-12 14:37:05.126853

<xarray.Dataset>
Dimensions:         (i: 36506)
Coordinates:
    time            (i) datetime64[ns] ...
    latitude        (i) float64 ...
    longitude       (i) float64 ...
    pressure        int64 -10
Dimensions without coordinates: i
Data variables:
    mean_temp       (i) float64 ...
    mean_sal        (i) float64 ...
    residuals_temp  (i) float64 ...
    residuals_sal   (i) float64 ...
Attributes:
    description:          Estimated mean temperature and salinity, and residu...
    pressureindex:        1
    number_of_harmonics:  2
    window_size1:         700000.0
    window_size2:         700000.0
    filename:             /global/homes/k/kefalc/code/anom_files/test/residua...
    created_date:         2025-05-12 14:37:16.793789

<xarray.Dataset>
Dimensions:         (i: 36506)
Coordinates:
    time            (i) datetime64[ns] ...
    latitude        (i) float64 ...
    longitude       (i) float64 ...
    pressure        int64 -100
Dimensions without coordinates: i
Data variables:
    mean_temp       (i) float64 ...
    mean_sal        (i) float64 ...
    residuals_temp  (i) float64 ...
    residuals_sal   (i) float64 ...
Attributes:
    description:          Estimated mean temperature and salinity, and residu...
    pressureindex:        2
    number_of_harmonics:  2
    window_size1:         700000.0
    window_size2:         700000.0
    filename:             /global/homes/k/kefalc/code/anom_files/test/residua...
    created_date:         2025-05-12 14:37:27.418626

<xarray.Dataset>
Dimensions:         (i: 36506)
Coordinates:
    time            (i) datetime64[ns] ...
    latitude        (i) float64 ...
    longitude       (i) float64 ...
    pressure        int64 -250
Dimensions without coordinates: i
Data variables:
    mean_temp       (i) float64 ...
    mean_sal        (i) float64 ...
    residuals_temp  (i) float64 ...
    residuals_sal   (i) float64 ...
Attributes:
    description:          Estimated mean temperature and salinity, and residu...
    pressureindex:        3
    number_of_harmonics:  2
    window_size1:         700000.0
    window_size2:         700000.0
    filename:             /global/homes/k/kefalc/code/anom_files/test/residua...
    created_date:         2025-05-12 14:37:37.350865

<xarray.Dataset>
Dimensions:         (i: 36506)
Coordinates:
    time            (i) datetime64[ns] ...
    latitude        (i) float64 ...
    longitude       (i) float64 ...
    pressure        int64 -500
Dimensions without coordinates: i
Data variables:
    mean_temp       (i) float64 ...
    mean_sal        (i) float64 ...
    residuals_temp  (i) float64 ...
    residuals_sal   (i) float64 ...
Attributes:
    description:          Estimated mean temperature and salinity, and residu...
    pressureindex:        4
    number_of_harmonics:  2
    window_size1:         700000.0
    window_size2:         700000.0
    filename:             /global/homes/k/kefalc/code/anom_files/test/residua...
    created_date:         2025-05-12 14:37:46.641501

<xarray.Dataset>
Dimensions:         (i: 36506)
Coordinates:
    time            (i) datetime64[ns] ...
    latitude        (i) float64 ...
    longitude       (i) float64 ...
    pressure        int64 -1000
Dimensions without coordinates: i
Data variables:
    mean_temp       (i) float64 ...
    mean_sal        (i) float64 ...
    residuals_temp  (i) float64 ...
    residuals_sal   (i) float64 ...
Attributes:
    description:          Estimated mean temperature and salinity, and residu...
    pressureindex:        5
    number_of_harmonics:  2
    window_size1:         700000.0
    window_size2:         700000.0
    filename:             /global/homes/k/kefalc/code/anom_files/test/residua...
    created_date:         2025-05-12 14:37:56.640034

<xarray.Dataset>
Dimensions:         (i: 36506)
Coordinates:
    time            (i) datetime64[ns] ...
    latitude        (i) float64 ...
    longitude       (i) float64 ...
    pressure        int64 -1500
Dimensions without coordinates: i
Data variables:
    mean_temp       (i) float64 ...
    mean_sal        (i) float64 ...
    residuals_temp  (i) float64 ...
    residuals_sal   (i) float64 ...
Attributes:
    description:          Estimated mean temperature and salinity, and residu...
    pressureindex:        6
    number_of_harmonics:  2
    window_size1:         700000.0
    window_size2:         700000.0
    filename:             /global/homes/k/kefalc/code/anom_files/test/residua...
    created_date:         2025-05-12 14:38:05.614672

In [28]:
##Remove matched indexes to select that data for TS only
for plevel in range (0,7):
    resfile = 'residualfile_plevel'+str(plevel)+'_dates'+str(time_i)+'_'+str(time_f)+'_test0.nc'
    dm = xr.open_dataset(mean_root +'final/TS/res/' + resfile)
    #display(dm)
    dm_new = dm.drop_isel(i=index_ts)
    #da.where(da.sum(dim='y') <= 20, drop=True)
    dm_new = dm_new.squeeze('pressure')
    #dm_new = dm_new.drop_dims('pressure')

    dm_new.to_netcdf(root + 'tsonly_forbgc_residualfile_plevel'+str(plevel)+'_dates'+str(time_i)+'_'+str(time_f)+'_test0.nc')

In [24]:
resfile = 'residualfile_plevel'+str(0)+'_dates'+str(time_i)+'_'+str(time_f)+'_test0.nc'
dm = xr.open_dataset(mean_root +'final/TS/res/' + resfile)

In [27]:
dm_new = dm.drop_isel(i=index_ts)
dm_new

<xarray.Dataset>
Dimensions:         (i: 202767, pressure: 1)
Coordinates:
    time            (i) datetime64[ns] ...
    latitude        (i) float64 ...
    longitude       (i) float64 ...
  * pressure        (pressure) int64 -5
Dimensions without coordinates: i
Data variables:
    mean_temp       (i, pressure) float64 ...
    mean_sal        (i, pressure) float64 ...
    residuals_temp  (i, pressure) float64 ...
    residuals_sal   (i, pressure) float64 ...
Attributes:
    description:          Estimated mean temperature and salinity, and residu...
    pressureindex:        0
    number_of_harmonics:  2
    window_size1:         700000.0
    window_size2:         700000.0
    filename:             /global/homes/k/kefalc/code/anom_files/test/residua...
    created_date:         2025-05-12 14:37:05.126853